In [65]:
# Define your CSV file
csv_file = '2024_US_craigslist.csv'  # Updated file name

# Reset the CSV file and DataFrame at the start
if os.path.exists(csv_file):
    os.remove(csv_file)

df = pd.DataFrame()

# Ensure that 'Page' column is numeric and initialize the last scraped page to 0
df['Page'] = np.nan
last_scraped_page = 0

print(f"Last scraped page: {last_scraped_page}")

Last scraped page: 0


In [79]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
import time
import random
from requests.adapters import HTTPAdapter, Retry
import os

# Define your CSV file
csv_file = '2024_US_craigslist.csv'  # Updated file name

# Initialize the DataFrame
df_craigslist = pd.DataFrame(columns=['post id', 'condition:', 'cylinders:', 'drive:', 'fuel:', 'odometer:', 
                                      'title status:', 'transmission:', 'type:', 'Ad Date', 'price', 
                                      'Make', 'Model', 'Year', 'Scraped on', 'Ad Age (days)', 'Page'])

print("DataFrame reset.")

car_urls = [
    # Add the exact car URLs you want to scrape here
    "https://losangeles.craigslist.org/search/cta#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=mercedes%20c#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=mercedes%20e#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=mercedes%20g#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=bmw%203#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=bmw%205#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=bmw%207#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=bmw%20x3#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=bmw%20x5#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=honda#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=honda%20accord#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=honda%20civic#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=honda%20cr#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=toyota#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=toyota%20avalon#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=toyota%20camry#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=toyota%20corolla#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=toyota%20prius#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=toyota%20rav4#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=toyota%20sienna#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=toyota%20tacoma#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=toyota%20yaris#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=vw#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=vw%20golf#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=vw%20tiguan#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=vw%20jetta#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=vw%20passat#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=volkswagen%20passat#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=volkswagen%20golf#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=volkswagen%20jetta#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=volkswagen%20touareg#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=ferrari#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=rolls%20royce#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=lamborghini#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=jaguar#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=audi#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=audi%20a3#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=audi%20a4#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=audi%20a5#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=audi%20a6#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=audi%20a8#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=audi%20q5#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=audi%20q7#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=fiat#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=fiat%20500#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=kia#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=kia%20optima#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=kia%20rio#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=kia%20sorento#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=kia%20soul#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=kia%20sportage#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=hyundai#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=hyundai%20accent#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=hyundai%20elantra#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=hyundai%20santa%20fe#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=hyundai%20sonata#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=tesla#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=ford#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=ford%20escape#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=ford%20explorer#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=ford%20f-150#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=ford%20focus#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=ford%20mustang&min_auto_year=2010#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=mini%20cooper#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=range%20rover#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=land%20rover#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=jeep#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=jeep%20cherokee#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=jeep%20wrangler#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=alfa%20romeo#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=chrysler#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=nissan#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=nissan%20altima#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=nissan%20murano#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=nissan%20rogue#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=nissan%20sentra#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=nissan%20versa#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=volvo&max_price=15000#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=chevrolet&max_price=15000#search=1~gallery~0~0",    
    "https://losangeles.craigslist.org/search/cta?auto_make_model=maserati#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=corvette#search=1~gallery~0~0",
    "https://losangeles.craigslist.org/search/cta?auto_make_model=maserati#search=1~gallery~0~0"    
]

# List of user agents to rotate
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/89.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
]

# Optional: List of proxies
proxies = [
    # "http://proxy1.example.com:8080",
    # "http://proxy2.example.com:8080",
]

# Define function to create a new session
def create_new_session():
    session = requests.Session()
    # Setup retry strategy for requests
    retry_strategy = Retry(
        total=5,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["HEAD", "GET", "OPTIONS"]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("https://", adapter)
    session.mount("http://", adapter)
    return session

# Initialize session and reset cookies before starting
http = create_new_session()
print("Initial session created and cookies reset.")

# Define function to get car listings URLs from a page
def get_car_urls(page_url):
    print(f"Scraping page URL: {page_url}")  # Debugging line to print the page URL
    try:
        headers = {
            "User-Agent": random.choice(user_agents)
        }
        proxy = {"http": random.choice(proxies)} if proxies else None
        response = http.get(page_url, headers=headers, proxies=proxy)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        car_elements = soup.select("li.cl-static-search-result a")
        car_urls = [{'url': elem['href'], 'post id': elem['href'].split('/')[-1].split('.')[0]} for elem in car_elements]
        print(f"Found {len(car_urls)} car URLs on page {page_url}")
        return car_urls
    except requests.exceptions.RequestException as e:
        print(f"Error fetching page {page_url}: {e}")
        return []

# Define function to scrape individual car details
def scrape_car_details(car):
    try:
        headers = {
            "User-Agent": random.choice(user_agents)
        }
        proxy = {"http": random.choice(proxies)} if proxies else None
        response = http.get(car['url'], headers=headers, proxies=proxy)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")

        car_data = {}

        # Extracting post ID
        car_data["post id"] = car['post id']

        # Extracting attribute group data
        attrgroup = soup.find_all("div", class_="attrgroup")
        for group in attrgroup:
            for div in group.find_all("div"):
                if div.find("span", class_="labl"):
                    key = div.find("span", class_="labl").get_text(strip=True).lower()
                    value = div.find("span", class_="valu").get_text(strip=True)
                    car_data[key] = value

        # Extracting year
        year_element = soup.find("span", class_="valu year")
        if year_element:
            car_data["Year"] = year_element.get_text(strip=True)

        # Extracting posting info data
        posting_info = soup.find_all("p", class_="postinginfo")
        for info in posting_info:
            if "post id" in info.get_text(strip=True):
                car_data["post id"] = info.get_text(strip=True).split(": ")[1]
            elif "posted:" in info.get_text(strip=True):
                car_data["Ad Date"] = info.find("time")["datetime"].split("T")[0]

        # Extracting price
        price_element = soup.find("span", class_="price")
        if price_element:
            car_data["price"] = price_element.get_text(strip=True).replace("$", "").replace(",", "")

        # Extracting make and model
        make_model_element = soup.find("span", class_="valu makemodel")
        if make_model_element:
            make_model_text = make_model_element.get_text(strip=True)
            # Handle exceptions
            exceptions = ["Rolls Royce", "Range Rover", "Mercedes Benz", "Land Rover", "Mercedes-Benz", "Rolls-Royce"]
            for exc in exceptions:
                if exc in make_model_text:
                    car_data["Make"] = exc
                    car_data["Model"] = make_model_text.replace(exc, "").strip()
                    break
            else:
                make_model_split = make_model_text.split()
                car_data["Make"] = make_model_split[0]
                car_data["Model"] = " ".join(make_model_split[1:])

        # Adding scraped date and calculating ad age
        car_data["Scraped on"] = datetime.now().strftime("%Y-%m-%d")
        if "Ad Date" in car_data:
            car_data["Ad Age (days)"] = (datetime.strptime(car_data["Scraped on"], "%Y-%m-%d") - 
                                         datetime.strptime(car_data["Ad Date"], "%Y-%m-%d")).days
        else:
            car_data["Ad Age (days)"] = None

        print(f"Scraped data for car: {car_data}")
        return car_data
    except requests.exceptions.RequestException as e:
        print(f"Error scraping car details from {car['url']}: {e}")
        return None

# Define function to scrape all car data
def scrape_all_cars(car_urls):
    global df_craigslist
    all_car_data = []  # Initialize all_car_data for each run
    for page_url in car_urls:
        car_urls_on_page = get_car_urls(page_url)
        for car in car_urls_on_page:
            try:
                car_data = scrape_car_details(car)
                if car_data:
                    # Check if the car with the same post id is already in the DataFrame
                    if car_data["post id"] in df_craigslist["post id"].values:
                        print(f"Skipping already scraped car with post id: {car_data['post id']}")
                        continue
                    all_car_data.append(car_data)
            except requests.exceptions.RequestException as e:
                print(f"Request failed for {car['url']}: {e}")
                continue  # Skip this link and move to the next one
        time.sleep(random.uniform(5, 10))  # Add a delay to avoid overwhelming the server
    if all_car_data:
        df_new = pd.DataFrame(all_car_data)
        df_craigslist = pd.concat([df_craigslist, df_new], ignore_index=True)
        df_craigslist.drop_duplicates(subset='post id', keep='last', inplace=True)
        df_craigslist.to_csv(csv_file, index=False)
        print(f"Data saved to {csv_file}.")
    return df_craigslist

# Scrape car data from the provided list of page URLs
df_craigslist = scrape_all_cars(car_urls)

# Print the number of rows in the DataFrame to verify
print(f"Total number of rows scraped: {len(df_craigslist)}")


DataFrame reset.
Initial session created and cookies reset.
Scraping page URL: https://losangeles.craigslist.org/search/cta#search=1~gallery~0~0
Found 354 car URLs on page https://losangeles.craigslist.org/search/cta#search=1~gallery~0~0
Scraped data for car: {'post id': '7753469442', 'condition:': 'like new', 'cylinders:': '4 cylinders', 'drive:': 'rwd', 'fuel:': 'gas', 'odometer:': '92000', 'paint color:': 'white', 'title status:': 'clean', 'transmission:': 'automatic', 'type:': 'sedan', 'Year': '2015', 'Ad Date': '2024-06-03', 'price': '10400', 'Make': 'bmw', 'Model': '328i', 'Scraped on': '2024-06-03', 'Ad Age (days)': 0}
Scraped data for car: {'post id': '7753469091', 'condition:': 'excellent', 'cylinders:': '4 cylinders', 'drive:': '4wd', 'fuel:': 'gas', 'odometer:': '121000', 'paint color:': 'blue', 'title status:': 'clean', 'transmission:': 'automatic', 'type:': 'hatchback', 'Year': '2010', 'Ad Date': '2024-06-03', 'price': '8500', 'Make': 'subaru', 'Model': 'forester', 'Scrape

Scraped data for car: {'post id': '7753462844', 'condition:': 'excellent', 'cylinders:': '4 cylinders', 'drive:': 'fwd', 'fuel:': 'hybrid', 'odometer:': '155000', 'paint color:': 'white', 'title status:': 'clean', 'transmission:': 'automatic', 'type:': 'hatchback', 'Year': '2010', 'Ad Date': '2024-06-03', 'price': '7800', 'Make': 'toyota', 'Model': 'prius', 'Scraped on': '2024-06-03', 'Ad Age (days)': 0}
Scraped data for car: {'post id': '7753462175', 'condition:': 'like new', 'cylinders:': '3 cylinders', 'drive:': 'fwd', 'fuel:': 'gas', 'odometer:': '74000', 'paint color:': 'red', 'title status:': 'clean', 'transmission:': 'automatic', 'type:': 'hatchback', 'Year': '2021', 'Ad Date': '2024-06-03', 'price': '7999', 'Make': 'mitsubishi', 'Model': 'mirage', 'Scraped on': '2024-06-03', 'Ad Age (days)': 0}
Scraped data for car: {'post id': '7753461512', 'vin:': 'WDCGG5HB5EG165175', 'condition:': 'like new', 'cylinders:': '6 cylinders', 'fuel:': 'gas', 'odometer:': '49500', 'paint color:': 

Scraped data for car: {'post id': '7753450830', 'fuel:': 'gas', 'odometer:': '45000', 'title status:': 'salvage', 'transmission:': 'automatic', 'Year': '2013', 'Ad Date': '2024-06-03', 'price': '26500', 'Make': 'a', 'Model': '', 'Scraped on': '2024-06-03', 'Ad Age (days)': 0}
Scraped data for car: {'post id': '7753450464', 'vin:': 'WDDGJ4HB6DG086104', 'fuel:': 'gas', 'odometer:': '87419', 'paint color:': 'white', 'title status:': 'clean', 'transmission:': 'automatic', 'type:': 'coupe', 'Year': '2013', 'Ad Date': '2024-06-03', 'price': '9988', 'Make': 'MERCEDES-BENZ', 'Model': 'C-CLASS', 'Scraped on': '2024-06-03', 'Ad Age (days)': 0}
Scraped data for car: {'post id': '7753450258', 'vin:': '1G1YR3DW5B5111229', 'condition:': 'excellent', 'cylinders:': '8 cylinders', 'drive:': 'rwd', 'fuel:': 'gas', 'odometer:': '29031', 'paint color:': 'black', 'title status:': 'clean', 'transmission:': 'manual', 'type:': 'convertible', 'Year': '2011', 'Ad Date': '2024-06-03', 'price': '44500', 'Make': '

Scraped data for car: {'post id': '7753438391', 'vin:': '1N4AL3AP5EC324713', 'condition:': 'good', 'cylinders:': '4 cylinders', 'drive:': 'fwd', 'fuel:': 'gas', 'odometer:': '194750', 'paint color:': 'grey', 'title status:': 'clean', 'transmission:': 'automatic', 'type:': 'sedan', 'Year': '2014', 'Ad Date': '2024-06-03', 'price': '4995', 'Make': 'Nissan', 'Model': 'Altima', 'Scraped on': '2024-06-03', 'Ad Age (days)': 0}
Scraped data for car: {'post id': '7753438223', 'vin:': '4JGCB6FE9BA128369', 'condition:': 'excellent', 'cylinders:': '6 cylinders', 'drive:': '4wd', 'fuel:': 'gas', 'odometer:': '150685', 'paint color:': 'black', 'title status:': 'clean', 'transmission:': 'automatic', 'type:': 'SUV', 'Year': '2011', 'Ad Date': '2024-06-03', 'price': '5995', 'Make': 'Mercedes-Benz', 'Model': 'R-Class', 'Scraped on': '2024-06-03', 'Ad Age (days)': 0}
Scraped data for car: {'post id': '7753437830', 'vin:': '1D7HA18226S628253', 'condition:': 'good', 'cylinders:': '8 cylinders', 'drive:': 

Scraped data for car: {'post id': '7753432468', 'vin:': '4S4BSBNCXF3247451', 'cylinders:': '4 cylinders', 'drive:': '4wd', 'fuel:': 'gas', 'odometer:': '133262', 'paint color:': 'black', 'title status:': 'clean', 'transmission:': 'automatic', 'type:': 'wagon', 'Year': '2015', 'Ad Date': '2024-06-03', 'price': '11995', 'Make': 'Subaru', 'Model': 'Outback 2.5i Limited Detail Is Pending See MANAGER/ More Pictures coming SOON', 'Scraped on': '2024-06-03', 'Ad Age (days)': 0}
Scraped data for car: {'post id': '7753432426', 'vin:': 'JTDKB20U687743394', 'cylinders:': '4 cylinders', 'drive:': 'fwd', 'fuel:': 'gas', 'odometer:': '122853', 'paint color:': 'blue', 'title status:': 'clean', 'transmission:': 'automatic', 'type:': 'hatchback', 'Year': '2008', 'Ad Date': '2024-06-03', 'price': '8995', 'Make': 'Toyota', 'Model': 'Prius', 'Scraped on': '2024-06-03', 'Ad Age (days)': 0}
Scraped data for car: {'post id': '7753432226', 'condition:': 'good', 'cylinders:': '6 cylinders', 'drive:': 'rwd', 'f

...

Scraped data for car: {'post id': '7752772251', 'condition:': 'excellent', 'fuel:': 'gas', 'odometer:': '78066', 'title status:': 'clean', 'transmission:': 'automatic', 'Year': '1975', 'Ad Date': '2024-06-01', 'price': '15995', 'Make': 'chevy', 'Model': 'corvette', 'Scraped on': '2024-06-04', 'Ad Age (days)': 3}
Scraped data for car: {'post id': '7752569525', 'condition:': 'excellent', 'fuel:': 'gas', 'odometer:': '88900', 'title status:': 'clean', 'transmission:': 'manual', 'Year': '1963', 'Ad Date': '2024-05-31', 'price': '159990', 'Make': 'Chevrolet', 'Model': 'Corvette', 'Scraped on': '2024-06-04', 'Ad Age (days)': 4}
Scraped data for car: {'post id': '7752568637', 'vin:': '1G1YB2D47P5138842', 'condition:': 'like new', 'cylinders:': '8 cylinders', 'drive:': 'rwd', 'fuel:': 'gas', 'odometer:': '449', 'paint color:': 'grey', 'title status:': 'clean', 'transmission:': 'automatic', 'type:': 'coupe', 'Year': '2023', 'Ad Date': '2024-05-31', 'price': '83999', 'Make': 'Chevrolet', 'Model'

Scraped data for car: {'post id': '7750781122', 'condition:': 'like new', 'cylinders:': '8 cylinders', 'drive:': 'rwd', 'fuel:': 'gas', 'odometer:': '76228', 'paint color:': 'black', 'title status:': 'clean', 'transmission:': 'manual', 'type:': 'coupe', 'Year': '1989', 'Ad Date': '2024-05-26', 'price': '18999', 'Make': 'CORVETTE', 'Model': 'C4', 'Scraped on': '2024-06-04', 'Ad Age (days)': 9}
Scraped data for car: {'post id': '7750649998', 'fuel:': 'gas', 'odometer:': '25580', 'title status:': 'clean', 'transmission:': 'automatic', 'Year': '2014', 'Ad Date': '2024-05-25', 'price': '43000', 'Make': 'chevy', 'Model': 'corvette', 'Scraped on': '2024-06-04', 'Ad Age (days)': 10}
Scraped data for car: {'post id': '7750495734', 'condition:': 'excellent', 'fuel:': 'gas', 'odometer:': '78066', 'title status:': 'clean', 'transmission:': 'automatic', 'Year': '1975', 'Ad Date': '2024-05-25', 'price': '13995', 'Make': 'chevy', 'Model': 'corvette', 'Scraped on': '2024-06-04', 'Ad Age (days)': 10}
S

Scraped data for car: {'post id': '7748158195', 'vin:': '1G1YG2D78H5113848', 'condition:': 'like new', 'cylinders:': '8 cylinders', 'drive:': 'rwd', 'fuel:': 'gas', 'odometer:': '26495', 'paint color:': 'silver', 'title status:': 'clean', 'transmission:': 'manual', 'type:': 'coupe', 'Year': '2017', 'Ad Date': '2024-05-18', 'price': '39995', 'Make': 'Chevrolet', 'Model': 'Corvette', 'Scraped on': '2024-06-04', 'Ad Age (days)': 17}
Scraped data for car: {'post id': '7748060929', 'condition:': 'like new', 'cylinders:': '8 cylinders', 'fuel:': 'gas', 'odometer:': '148264', 'paint color:': 'red', 'title status:': 'clean', 'transmission:': 'manual', 'type:': 'coupe', 'Year': '1995', 'Ad Date': '2024-05-17', 'price': '7000', 'Make': 'chevy', 'Model': 'corvette', 'Scraped on': '2024-06-04', 'Ad Age (days)': 18}
Scraped data for car: {'post id': '7748060442', 'condition:': 'excellent', 'cylinders:': '8 cylinders', 'drive:': 'fwd', 'fuel:': 'gas', 'odometer:': '37223', 'paint color:': 'blue', 't

Scraped data for car: {'post id': '7746697028', 'condition:': 'excellent', 'cylinders:': '8 cylinders', 'drive:': 'rwd', 'fuel:': 'gas', 'odometer:': '95485', 'paint color:': 'grey', 'title status:': 'clean', 'transmission:': 'manual', 'type:': 'coupe', 'Year': '2007', 'Ad Date': '2024-05-13', 'price': '19299', 'Make': 'chevy', 'Model': 'corvette convertible', 'Scraped on': '2024-06-04', 'Ad Age (days)': 22}
Scraped data for car: {'post id': '7746593437', 'vin:': '1G1YE2D32P5604692', 'condition:': 'good', 'cylinders:': '8 cylinders', 'drive:': 'rwd', 'fuel:': 'gas', 'odometer:': '63', 'paint color:': 'silver', 'title status:': 'clean', 'transmission:': 'other', 'type:': 'coupe', 'Year': '2023', 'Ad Date': '2024-05-13', 'price': '189900', 'Make': 'Chevrolet', 'Model': 'Corvette', 'Scraped on': '2024-06-04', 'Ad Age (days)': 22}
Scraped data for car: {'post id': '7746234793', 'vin:': '194676S125369', 'condition:': 'excellent', 'cylinders:': 'other', 'fuel:': 'gas', 'odometer:': '', 'titl

Scraped data for car: {'post id': '7752476115', 'condition:': 'excellent', 'fuel:': 'gas', 'odometer:': '36000', 'title status:': 'clean', 'transmission:': 'manual', 'Year': '2018', 'Ad Date': '2024-05-31', 'price': '31880', 'Make': 'maserati', 'Model': 'ghibli s q4', 'Scraped on': '2024-06-04', 'Ad Age (days)': 4}
Scraped data for car: {'post id': '7751546551', 'condition:': 'excellent', 'cylinders:': '8 cylinders', 'drive:': 'rwd', 'fuel:': 'gas', 'odometer:': '51200', 'paint color:': 'black', 'title status:': 'clean', 'transmission:': 'automatic', 'Year': '2005', 'Ad Date': '2024-05-28', 'price': '17900', 'Make': 'maserati', 'Model': 'gransport', 'Scraped on': '2024-06-04', 'Ad Age (days)': 7}
Scraped data for car: {'post id': '7751510262', 'fuel:': 'gas', 'odometer:': '68000', 'title status:': 'clean', 'transmission:': 'other', 'Year': '2017', 'Ad Date': '2024-05-28', 'price': '32000', 'Make': 'maserati', 'Model': 'levante', 'Scraped on': '2024-06-04', 'Ad Age (days)': 7}
Scraped d